In [1]:
#TO DO

# 1. Implementar ROC al hacer CrossValidation
# 2. Implementar las demas metricas para el mejor modelo de cada clase
# 3. Revisar CrossValidation en DecisionTree y NaiveBayes
# 4. Revisar errores en SVM Lineal y LogisticRegression
# 5. Completar preprocesamientos (valores nulos, encoding, scaling, seleccion)
# 6. Revisar y completar la parte de ensambles
# 7. Implementar Red Neuronal
# 8. Implementar Cascading

# 9. Separar modelos en respectivos notebooks
# 10. Predecir sobre los datos nuevos

In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure
import matplotlib.gridspec as gridspec

from sklearn import neighbors, datasets
from sklearn import naive_bayes
from sklearn import svm
from sklearn import tree

from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from sklearn import model_selection
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_score, recall_score, f1_score

pd.set_option('display.max_colwidth', None)
    

# Datos

In [3]:
data = pd.read_csv('tp-2020-2c-train-cols2.csv')
targets = pd.read_csv('tp-2020-2c-train-cols1.csv')

In [4]:
df = (pd.merge(left=data, right=targets, how="left", on="id_usuario", validate="one_to_one"))

In [5]:
df.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede,volveria
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes,0
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes,0
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita,0
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo,0
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo,1


# Preprocesamiento

In [6]:
encoding_params = {'onehot_drop':'first'
                  }
scaling_params = {'standard_withmean':True}
selection_params = {'vt_threshold':0,
                    'rfe_estimator':'estimator'}

columnas_a_encodear = ['tipo_de_sala', 'genero', 'nombre_sede']

In [7]:
#encoders
def OrdinalEncoderWrapper(encoding_params):
    return OrdinalEncoder()

def OneHotWrapper(encoding_params):
    return OneHotEncoder(drop=encoding_params['onehot_drop'])

def LabelEncoderWrapper(encoding_params):
    return LabelEncoder()

encoders = {'ordinal': OrdinalEncoderWrapper(encoding_params),
           'label': LabelEncoderWrapper(encoding_params),
            'onehot': OneHotWrapper(encoding_params)
           }

#scalers
def StandardScalerWrapper(scaling_params):
    return StandardScaler(with_mean=scaling_params['standard_withmean'])

def MinMaxScalerWrapper(scaling_params):
    return MinMaxScaler()

def RobustScalerWrapper(scaling_params):
    return RobustScaler()

def PowerTransformerWrapper(scaling_params):
    return PowerTransformer()

def NormalizerWrapper(scaling_params):
    return Normalizer()

scalers = {'standard': StandardScalerWrapper(scaling_params),
          'minmax': MinMaxScalerWrapper(scaling_params),
          'robust': RobustScalerWrapper(scaling_params),
           'power' : PowerTransformerWrapper(scaling_params),
           'normalizer' : NormalizerWrapper(scaling_params)
          }

#Selectors


def VarianceThresholdWrapper(selection_params):
    return VarianceThreshold()

def RFEWrapper(selection_params):
    return RFE(selection_params['rfe_estimator'])

def FeatureHasherWrapper(selection_params):
    return FeatureHasher()


selectors = {'var_thres': VarianceThresholdWrapper(selection_params),
            'rfe': RFEWrapper(selection_params),
            'feature_hasher': FeatureHasherWrapper(selection_params)}

In [8]:
#Funciones auxiliares
#robado de la practica
def droppear_nulos_por_columna(data):
    NULL_REMOVE_PCT = 0.30
    cols = data.isna().mean()
    cols = cols[cols < NULL_REMOVE_PCT]
    return data[cols.index]

def droppear_filas_sin_sede(data):
    _data = data.drop(data.loc[data['nombre_sede'].isna()].index, inplace=False)
    _data.reset_index(drop=True)
    return _data
    

def prepro_1(X, y, encoder, scaler, selector):
    _X = X.copy(deep=True)
    _y = y.copy(deep=True)
    
    #tratamiento de nulos
    _X = droppear_nulos_por_columna(_X)
    #_X = droppear_filas_sin_sede(_X) rompe el dataframe, no se por que
    _X.drop(['id_ticket','nombre','id_usuario'], axis=1, inplace=True)
    _X['edad'] = SimpleImputer(strategy='median').fit_transform(_X[['edad']])
    
    
    #encoding
    _encoder = encoders[encoder]
    if(encoder == 'onehot'):
        for col in columnas_a_encodear:
            
            encoded = _encoder.fit(_X[[col]].astype(str))
            categories = list(encoded.categories_)
            encoded = encoded.transform(_X[[col]].astype(str)).todense().astype(int)
            encoded = pd.DataFrame(encoded)
            categories = np.delete(categories, 0)
            encoded.columns = categories
            
            _X = pd.concat([_X, encoded], axis=1)
            _X.drop(labels=col, axis=1, inplace=True)
    
    #seleccion
    _selector = selectors[selector]
    #_X = _selector.fit_transform(_X,_y)
    
    #train-test split
    X_train, X_test, y_train, y_test = model_selection.train_test_split(_X, _y, test_size=0.2, random_state=7, stratify=y['volveria'])
    
    #escalado
    if(scaler != 'none'):
        _scaler = scalers[scaler]
        X_train = _scaler.fit_transform(X_train, y_train)
        X_test = _scaler.transform(X_test)
    
    y_train = y_train['volveria'].to_numpy(copy=True)
    y_test = y_test['volveria'].to_numpy(copy=True)
    
    return X_train, X_test, y_train, y_test


In [55]:
X_train_a, X_test_a, y_train_a, y_test_a = prepro_1(data, targets, 'onehot', 'standard', 'var_thres')
X_train_b, X_test_b, y_train_b, y_test_b = prepro_1(data, targets, 'onehot', 'minmax', 'var_thres')
X_train_c, X_test_c, y_train_c, y_test_c = prepro_1(data, targets, 'onehot', 'robust', 'var_thres')
X_train_d, X_test_d, y_train_d, y_test_d = prepro_1(data, targets, 'onehot', 'power', 'var_thres')
X_train_e, X_test_e, y_train_e, y_test_e = prepro_1(data, targets, 'onehot', 'normalizer', 'var_thres')

datos = [
    [X_train_a, X_test_a, y_train_a, y_test_a, 'onehot', 'standard', 'var_thres'],
    [X_train_b, X_test_b, y_train_b, y_test_b, 'onehot', 'minmax', 'var_thres'],
    [X_train_c, X_test_c, y_train_c, y_test_c, 'onehot', 'robust', 'var_thres'],
    [X_train_d, X_test_d, y_train_d, y_test_d, 'onehot', 'power', 'var_thres'],
    [X_train_e, X_test_e, y_train_e, y_test_e, 'onehot', 'normalizer', 'var_thres']
]

datos_nb = [
    [X_train_b, X_test_b, y_train_b, y_test_b, 'onehot', 'minmax', 'var_thres'],
    [X_train_e, X_test_e, y_train_e, y_test_e, 'onehot', 'normalizer', 'var_thres']
]

mejores_modelos = []

---

# Entrenando modelos

In [56]:
def aplicar_gscv(gscv, columnas):
    metricas = []
    mejor_roc_auc = -1
    mejor_modelo = None
    prepro_mejor_modelo = None
    info_mejor_modelo = None
    
    for conjunto in datos: #pruebo 5 conjuntos de datos, cada uno con distintos preprocesamientos
    
        gridsearch_resultados = gscv.fit(conjunto[0], conjunto[2]) #X_train e y_train, respectivamente
        row = []
        row.append(conjunto[4]) #encoding
        row.append(conjunto[5]) #scaling
        row.append(conjunto[6]) #selection
        
        for key in gridsearch_resultados.best_params_:
            row.append(gridsearch_resultados.best_params_[key]) #Agregar el mejor valor de cada parametro testeado
        row.append(gridsearch_resultados.best_score_)           #Agregar mejor score
        row.append(gridsearch_resultados.score(conjunto[1], conjunto[3]))  #Agregar score con holdout
        
        #roc_auc
        roc_auc = roc_auc_score(conjunto[3], gridsearch_resultados.best_estimator_.predict_proba(conjunto[1])[:, 1])
        
        if(roc_auc > mejor_roc_auc):
            mejor_roc_auc = roc_auc
            mejor_modelo = gridsearch_resultados.best_estimator_
            prepro_mejor_modelo = conjunto[4] + conjunto[5] + conjunto[6]
        
        row.append(roc_auc)
        metricas.append(row)
        
    info_mejor_modelo = [mejor_modelo, prepro_mejor_modelo, mejor_roc_auc]
    mejores_modelos.append(info_mejor_modelo)
    
    resultados = pd.DataFrame(metricas, columns=columnas)
    resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
    display(resultados)

In [57]:
def aplicar_gscv_nb(gscv, columnas):
    metricas = []
    mejor_roc_auc = -1
    mejor_modelo = None
    prepro_mejor_modelo = None
    
    for conjunto in datos_nb: #pruebo 5 conjuntos de datos, cada uno con distintos preprocesamientos
    
        gridsearch_resultados = gscv.fit(conjunto[0], conjunto[2]) #X_train e y_train, respectivamente
        row = []
        row.append(conjunto[4]) #encoding
        row.append(conjunto[5]) #scaling
        row.append(conjunto[6]) #selection
        
        for key in gridsearch_resultados.best_params_:
            row.append(gridsearch_resultados.best_params_[key]) #Agregar el mejor valor de cada parametro testeado
        row.append(gridsearch_resultados.best_score_)           #Agregar mejor score
        row.append(gridsearch_resultados.score(conjunto[1], conjunto[3]))  #Agregar score con holdout
        
        #roc_auc
        roc_auc = roc_auc_score(conjunto[3], gridsearch_resultados.best_estimator_.predict_proba(conjunto[1])[:, 1])
        
        #if(roc_auc > mejor_roc_auc):
        #    mejor_roc_auc = roc_auc
        #    mejor_modelo = gridsearch_resultados.best_estimator_
        
        
        row.append(roc_auc)
        
        mejores_modelos.append(gridsearch_resultados.best_estimator_)
        
        metricas.append(row)
    
    #mejores_modelos.append(mejor_modelo)
    resultados = pd.DataFrame(metricas, columns=columnas)
    resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
    display(resultados)

### 3.1 KNN

Entrenamiento con valores por defecto.

In [58]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_knn = neighbors.KNeighborsClassifier()
    clf_knn.fit(conjunto[0], conjunto[2])
    pred = clf_knn.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_knn.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
3,onehot,power,var_thres,0.838509,0.875855
0,onehot,standard,var_thres,0.801242,0.849055
1,onehot,minmax,var_thres,0.801242,0.845960
4,onehot,normalizer,var_thres,0.788820,0.841235
2,onehot,robust,var_thres,0.795031,0.822336


---

Busqueda de hiperparámetros mediante GridSearch

In [59]:
clf_knn = neighbors.KNeighborsClassifier()

knn_grid_params = {
    'n_neighbors': [3,5,11,19],
    'weights' : ['distance', 'uniform'],
    'metric' : ['minkowski', 'euclidean', 'manhattan']
}

gs = model_selection.GridSearchCV(
    clf_knn,
    knn_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1,
    scoring='accuracy'
)

columnas=['encoding','scaling','selection','metric', 'n_neighbors', 'weights', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


,encoding,scaling,selection,metric,n_neighbors,weights,score,holdout_score,auroc_score
3,onehot,power,var_thres,manhattan,11,uniform,0.817187,0.801242,0.874552
0,onehot,standard,var_thres,manhattan,11,uniform,0.828125,0.770186,0.862170
4,onehot,normalizer,var_thres,manhattan,11,distance,0.803125,0.770186,0.845226
1,onehot,minmax,var_thres,minkowski,3,uniform,0.815625,0.819876,0.837814
2,onehot,robust,var_thres,manhattan,19,uniform,0.825000,0.795031,0.834474


---

In [100]:


def resumen_mejor_modelo(mdl, xtn, ytn, xts, yts):
    
    mdl.fit(xtn, ytn)
    pred = mdl.predict(xts)
    mdl_score = (yts == pred)
    
    mdl_f1 = f1_score(y_test_d, mdl.predict(xts))
    mdl_recall = recall_score(y_test_d, mdl.predict(xts))
    mdl_precision = precision_score(yts, mdl.predict(xts))
    mdl_roc_auc = roc_auc_score(yts, mdl.predict_proba(xts)[:, 1])    



    metricas_mejor_mdl = pd.DataFrame([[mdl,
                                    mdl_score.mean(),
                                    mdl_precision, 
                                    mdl_recall, 
                                    mdl_f1,
                                    mdl_roc_auc]],
                                    columns=['modelo','score','precision','recall','f1','auc_roc_score'])
    display(metricas_mejor_mdl)
    

In [101]:
mejor_knn = neighbors.KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan')

resumen_mejor_modelo(mejor_knn, X_train_d, y_train_d, X_test_d, y_test_d)

,modelo,score,precision,recall,f1,auc_roc_score
0,"KNeighborsClassifier(metric='manhattan', n_neighbors=11)",0.801242,0.767857,0.693548,0.728814,0.874552


In [91]:
mejor_knn = neighbors.KNeighborsClassifier(n_neighbors=11, weights='uniform', metric='manhattan')
mejor_knn.fit(X_train_d, y_train_d)
pred = mejor_knn.predict(X_test_d)
mejor_knn_score = (y_test_d == pred)
mejor_knn_precision = precision_score(y_test_d, mejor_knn.predict(X_test_d))
mejor_knn_recall = recall_score(y_test_d, mejor_knn.predict(X_test_d))
mejor_knn_f1 = f1_score(y_test_d, mejor_knn.predict(X_test_d))
mejor_knn_roc_auc = roc_auc_score(y_test_d, mejor_knn.predict_proba(X_test_d)[:, 1])    



metricas_mejor_knn = pd.DataFrame([[mejor_knn,
                                    mejor_knn_score.mean(),
                                    mejor_knn_precision, 
                                    mejor_knn_recall, 
                                    mejor_knn_f1,
                                    mejor_knn_roc_auc]],
                                    columns=['modelo','score','precision','recall','f1','auc_roc_score'])
metricas_mejor_knn

,modelo,score,precision,recall,f1,auc_roc_score
0,"KNeighborsClassifier(metric='manhattan', n_neighbors=11)",0.801242,0.767857,0.693548,0.728814,0.874552


---

### 3.2 Naive Bayes

#### 3.2.1 Gaussiano

In [15]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_nb_gauss = naive_bayes.GaussianNB()
    clf_nb_gauss.fit(conjunto[0], conjunto[2])
    pred = clf_nb_gauss.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_nb_gauss.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
0,onehot,standard,var_thres,0.645963,0.818589
1,onehot,minmax,var_thres,0.658385,0.818589
2,onehot,robust,var_thres,0.652174,0.818589
3,onehot,power,var_thres,0.614907,0.811909
4,onehot,normalizer,var_thres,0.664596,0.771017


---

In [103]:
mejor_naive_bayes = naive_bayes.GaussianNB()

resumen_mejor_modelo(mejor_naive_bayes, X_train_b, y_train_b, X_test_b, y_test_b)

,modelo,score,precision,recall,f1,auc_roc_score
0,GaussianNB(),0.658385,0.888889,0.129032,0.225352,0.818589


---

#### 3.2.2 Multinomial

In [16]:
#StandardScaler, RobustScaler y PowerTransformer arrojan valores negativos, no podemos usar el resultado en MultinomialNB

In [17]:
#Datos escalados con MinMaxScaler

clf_nb_mult = naive_bayes.MultinomialNB()
clf_nb_mult.fit(X_train_b, y_train_b)
pred = clf_nb_mult.predict(X_test_b)
score = (y_test_b == pred)
display("accuracy: ", score.mean())
display("auroc score: ", roc_auc_score(conjunto[3], clf_nb_mult.predict_proba(conjunto[1])[:, 1]))

'accuracy: '

0.7453416149068323

'auroc score: '

0.786493971977843

In [18]:
#Datos escalados con Normalizer

clf_nb_mult = naive_bayes.MultinomialNB(alpha=1.2)
clf_nb_mult.fit(X_train_e, y_train_e)
pred = clf_nb_mult.predict(X_test_e)
score = (y_test_e == pred)
display("accuracy: ", score.mean())
display("auroc score: ", roc_auc_score(conjunto[3], clf_nb_mult.predict_proba(conjunto[1])[:, 1]))

'accuracy: '

0.6149068322981367

'auroc score: '

0.7925219941348973

In [104]:
clf_nb_mult = naive_bayes.MultinomialNB()

nb_grid_params = {
    'alpha': [0.01, 0.1, 0.5, 0.9, 1, 1.2, 1.4, 1.6]
}

gs = model_selection.GridSearchCV(
    clf_nb_mult,
    nb_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1,
    scoring='accuracy'
)

columnas=['encoding','scaling','selection', 'alpha', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv_nb(gs, columnas)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits


,encoding,scaling,selection,alpha,score,holdout_score,auroc_score
0,onehot,minmax,var_thres,0.50,0.776563,0.745342,0.840094
1,onehot,normalizer,var_thres,0.01,0.621875,0.614907,0.794803


---

In [106]:
mejor_multinomial = naive_bayes.MultinomialNB()

resumen_mejor_modelo(mejor_multinomial, X_train_b, y_train_b, X_test_b, y_test_b)

,modelo,score,precision,recall,f1,auc_roc_score
0,MultinomialNB(),0.745342,0.690909,0.612903,0.649573,0.839769


---

### 3.3 SVM

#### 3.3.1 Lineal

In [20]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_linear_svc = svm.LinearSVC()
    clf_linear_svc.fit(conjunto[0], conjunto[2])
    pred = clf_linear_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    #row.append(roc_auc_score(conjunto[3], clf_linear_svc.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
#resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.788820
1,onehot,minmax,var_thres,0.788820
2,onehot,robust,var_thres,0.795031
3,onehot,power,var_thres,0.807453
4,onehot,normalizer,var_thres,0.695652


In [21]:
#Datos escalados con StandardScaler

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_a, y_train_a)
pred = clf_linear_svc.predict(X_test_a)
score = (y_test_a == pred)
score.mean()

/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7888198757763976

In [22]:
#Datos escalados con MinMaxScaler

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_b, y_train_b)
pred = clf_linear_svc.predict(X_test_b)
score = (y_test_b == pred)
score.mean()

0.7888198757763976

In [23]:
#Datos escalados con RobustScaler

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_c, y_train_c)
pred = clf_linear_svc.predict(X_test_c)
score = (y_test_c == pred)
score.mean()

/home/javier/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.7950310559006211

In [24]:
#Datos escalados con PowerTransformer

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_d, y_train_d)
pred = clf_linear_svc.predict(X_test_d)
score = (y_test_d == pred)
score.mean()

0.8074534161490683

In [25]:
#Datos escalados con Normalizer

clf_linear_svc = svm.LinearSVC()
clf_linear_svc.fit(X_train_e, y_train_e)
pred = clf_linear_svc.predict(X_test_e)
score = (y_test_e == pred)
score.mean()

0.6956521739130435

---

Busqueda de hiperparametros mediante GridSearch

In [26]:
clf_linear_svc = svm.LinearSVC()

svc_grid_params = {
    'C' : [0.01, 0.1, 1, 10],
    'penalty' : ['l1', 'l2']
}

gs = model_selection.GridSearchCV(
    clf_linear_svc,
    svc_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'penalty', 'score', 'holdout_score', 'auroc_score']
aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.80625         nan 0.80625         nan 0.8046875       nan
 0.80625  ]
  category=UserWarning


AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

---

#### 3.3.2 Polinomial

Entrenamiento con valores por defecto

In [27]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_svc = svm.SVC(kernel='poly', probability=True)
    clf_svc.fit(conjunto[0], conjunto[2])
    pred = clf_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_svc.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
3,onehot,power,var_thres,0.807453,0.858341
2,onehot,robust,var_thres,0.770186,0.848974
0,onehot,standard,var_thres,0.838509,0.827224
1,onehot,minmax,var_thres,0.826087,0.825269
4,onehot,normalizer,var_thres,0.708075,0.815331


---

Busqueda de hiperparametros mediante GridSearch

In [32]:
clf_svc = svm.SVC(kernel='poly')

svc_grid_params = {
    'C' : [0.01, 0.1, 1, 10],
    'degree': [2,3,4],
    'coef0': [0.0, 0.5, 1.0],
    'gamma': [1.0, 1.5, 2.0]
}

gs = model_selection.GridSearchCV(
    clf_svc,
    svc_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1
)



In [29]:
#Tardan mucho por la cantidad de fits que hay que hacer

In [31]:
#gs_results = gs.fit(X_train_a, y_train_a)
#display(gs_results.best_params_)
#display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_b, y_train_b)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_c, y_train_c)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_d, y_train_d)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

In [ ]:
gs_results = gs.fit(X_train_e, y_train_e)
display(gs_results.best_params_)
display(gs_results.best_score_)
#scores = model_selection.cross_val_score(clf_svc, df_X, df_y, cv=5)
#display(scores)

---

#### 3.3.3 Radial

Entrenamiento con valores por defecto

In [33]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_svc = svm.SVC(kernel='rbf', probability=True)
    clf_svc.fit(conjunto[0], conjunto[2])
    pred = clf_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_svc.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
3,onehot,power,var_thres,0.838509,0.877566
2,onehot,robust,var_thres,0.819876,0.875937
0,onehot,standard,var_thres,0.850932,0.860459
1,onehot,minmax,var_thres,0.826087,0.860134
4,onehot,normalizer,var_thres,0.683230,0.801971


---

Busqueda de hiperparametros mediante GridSearch

In [34]:
clf_svc = svm.SVC(kernel='rbf', probability=True)

svc_grid_params = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

gs = model_selection.GridSearchCV(
    clf_svc,
    svc_grid_params,
    verbose = 3,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'gamma', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 49 candidates, totalling 245 fits
Fitting 5 folds for each of 49 candidates, totalling 245 fits
Fitting 5 folds for each of 49 candidates, totalling 245 fits
Fitting 5 folds for each of 49 candidates, totalling 245 fits
Fitting 5 folds for each of 49 candidates, totalling 245 fits


,encoding,scaling,selection,C,gamma,score,holdout_score,auroc_score
3,onehot,power,var_thres,1,0.10,0.825000,0.838509,0.877566
4,onehot,normalizer,var_thres,10,10.00,0.806250,0.795031,0.855083
1,onehot,minmax,var_thres,10,1.00,0.829688,0.807453,0.836022
0,onehot,standard,var_thres,1000,0.01,0.826562,0.832298,0.823477
2,onehot,robust,var_thres,10,0.10,0.818750,0.838509,0.822010


---

#### 3.3.4 Sigmoide

Entrenamiento con valores por defecto

In [35]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_svc = svm.SVC(kernel='sigmoid', probability=True)
    clf_svc.fit(conjunto[0], conjunto[2])
    pred = clf_svc.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_svc.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score','auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
4,onehot,normalizer,var_thres,0.639752,0.789752
0,onehot,standard,var_thres,0.714286,0.755458
3,onehot,power,var_thres,0.683230,0.732323
1,onehot,minmax,var_thres,0.677019,0.694607
2,onehot,robust,var_thres,0.639752,0.641659


---

Busqueda de hiperparametros mediante GridSearch

In [36]:
clf_svc = svm.SVC(kernel='sigmoid', probability=True)

svc_grid_params = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'coef0': [0.0, 0.5, 1.0]
}

gs = model_selection.GridSearchCV(
    clf_svc,
    svc_grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'coef0', 'gamma', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 147 candidates, totalling 735 fits
Fitting 5 folds for each of 147 candidates, totalling 735 fits
Fitting 5 folds for each of 147 candidates, totalling 735 fits
Fitting 5 folds for each of 147 candidates, totalling 735 fits
Fitting 5 folds for each of 147 candidates, totalling 735 fits


,encoding,scaling,selection,C,coef0,gamma,score,holdout_score,auroc_score
1,onehot,minmax,var_thres,0.1,0.0,0.100,0.796875,0.795031,0.869909
2,onehot,robust,var_thres,100.0,0.0,0.001,0.793750,0.782609,0.852965
3,onehot,power,var_thres,0.1,0.0,0.100,0.787500,0.788820,0.825269
0,onehot,standard,var_thres,1000.0,0.5,0.001,0.800000,0.757764,0.820381
4,onehot,normalizer,var_thres,1000.0,0.0,0.100,0.789062,0.751553,0.818589


---

### 3.4 Regresor Lineal (Logistic Regression)

Entrenamiento con valores por defecto

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_logreg = LogisticRegression()
    clf_logreg.fit(conjunto[0], conjunto[2])
    pred = clf_logreg.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_logreg.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
3,onehot,power,var_thres,0.807453,0.865836
1,onehot,minmax,var_thres,0.801242,0.864695
0,onehot,standard,var_thres,0.801242,0.860459
2,onehot,robust,var_thres,0.801242,0.858993
4,onehot,normalizer,var_thres,0.670807,0.771017


---

Busqueda de Hiperparámetros mediante GridSearch

In [39]:
clf_logreg = LogisticRegression()

logreg_grid_params = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1','l2','elasticnet','none'],
}

gs = model_selection.GridSearchCV(
    clf_logreg,
    logreg_grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','C', 'penalty', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.6359375       nan 0.8109375       nan 0.8140625       nan
 0.8109375       nan 0.815625        nan 0.8109375       nan 0.8109375
       nan 0.8109375       nan 0.8109375       nan 0.8109375       nan
 0.8109375       nan 0.8109375       nan 0.8109375       nan 0.8109375]
  category=UserWarning


Fitting 5 folds for each of 28 candidates, totalling 140 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.6171875       nan 0.8109375       nan 0.7328125       nan
 0.8109375       nan 0.8234375       nan 0.8109375       nan 0.8046875
       nan 0.8109375       nan 0.8078125       nan 0.8109375       nan
 0.8109375       nan 0.8109375       nan 0.8109375       nan 0.8109375]
  category=UserWarning


Fitting 5 folds for each of 28 candidates, totalling 140 fits


...

Fitting 5 folds for each of 28 candidates, totalling 140 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.64375         nan 0.7984375       nan 0.7890625       nan
 0.7984375       nan 0.803125        nan 0.7984375       nan 0.7984375
       nan 0.7984375       nan 0.7984375       nan 0.7984375       nan
 0.7984375       nan 0.7984375       nan 0.7984375       nan 0.7984375]
  category=UserWarning


Fitting 5 folds for each of 28 candidates, totalling 140 fits


/home/javier/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [      nan 0.6171875       nan 0.7828125       nan 0.6171875       nan
 0.7828125       nan 0.621875        nan 0.7828125       nan 0.6859375
       nan 0.7828125       nan 0.734375        nan 0.7828125       nan
 0.7640625       nan 0.7828125       nan 0.7859375       nan 0.7828125]
  category=UserWarning
/home/javier/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,encoding,scaling,selection,C,penalty,score,holdout_score,auroc_score
3,onehot,power,var_thres,0.1,l2,0.803125,0.795031,0.867954
0,onehot,standard,var_thres,0.1,l2,0.815625,0.801242,0.862414
1,onehot,minmax,var_thres,0.1,l2,0.823438,0.801242,0.861437
2,onehot,robust,var_thres,0.1,l2,0.815625,0.807453,0.857527
4,onehot,normalizer,var_thres,1000.0,l2,0.785937,0.776398,0.822173


### 3.5 Decision Tree

Entrenamiento con valores por defecto

In [40]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_tree = tree.DecisionTreeClassifier(random_state=13)
    clf_tree.fit(conjunto[0], conjunto[2])
    pred = clf_tree.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_tree.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
2,onehot,robust,var_thres,0.813665,0.818671
0,onehot,standard,var_thres,0.807453,0.809466
1,onehot,minmax,var_thres,0.807453,0.808325
3,onehot,power,var_thres,0.801242,0.797084
4,onehot,normalizer,var_thres,0.770186,0.765559


---

Busqueda de Hiperparámetros mediante GridSearch

In [41]:
clf_tree = tree.DecisionTreeClassifier(random_state=13)

tree_grid_params = {
    'max_depth': np.arange(1,32),
    'min_samples_leaf': np.arange(1,32),
    'criterion': ["gini","entropy"]
}

gs = model_selection.GridSearchCV(
    clf_tree,
    tree_grid_params,
    verbose = 1,
    cv = 5,
    n_jobs = -1
)

columnas=['encoding','scaling','selection','criterion', 'max_depth', 'min_samples_leaf', 'score', 'holdout_score', 'auroc_score']

aplicar_gscv(gs, columnas)

Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits
Fitting 5 folds for each of 1922 candidates, totalling 9610 fits


,encoding,scaling,selection,criterion,max_depth,min_samples_leaf,score,holdout_score,auroc_score
1,onehot,minmax,var_thres,gini,6,2,0.817187,0.826087,0.846530
0,onehot,standard,var_thres,gini,8,3,0.815625,0.826087,0.841153
4,onehot,normalizer,var_thres,gini,4,10,0.818750,0.795031,0.836918
3,onehot,power,var_thres,gini,6,2,0.817187,0.819876,0.831460
2,onehot,robust,var_thres,entropy,4,10,0.815625,0.788820,0.831134


---

### 3.6 Redes Neuronales

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

### 3.7 Ensambles

In [42]:
from sklearn import ensemble

#### 3.7.1 Bagging

In [43]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_bagging = ensemble.BaggingClassifier()
    clf_bagging.fit(conjunto[0], conjunto[2])
    pred = clf_bagging.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_bagging.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
0,onehot,standard,var_thres,0.838509,0.867628
1,onehot,minmax,var_thres,0.819876,0.866813
3,onehot,power,var_thres,0.813665,0.852802
2,onehot,robust,var_thres,0.826087,0.850440
4,onehot,normalizer,var_thres,0.788820,0.846123


---

#### 3.7.2 RandomForest

In [44]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    random_forest = ensemble.RandomForestClassifier()
    random_forest.fit(conjunto[0], conjunto[2])
    pred = random_forest.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], random_forest.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score','auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
4,onehot,normalizer,var_thres,0.801242,0.872434
2,onehot,robust,var_thres,0.807453,0.869420
0,onehot,standard,var_thres,0.813665,0.865754
1,onehot,minmax,var_thres,0.813665,0.865510
3,onehot,power,var_thres,0.801242,0.858097


#### 3.7.3 Gradient Boost

In [45]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_boosting = ensemble.GradientBoostingClassifier()
    clf_boosting.fit(conjunto[0], conjunto[2])
    pred = clf_boosting.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    row.append(roc_auc_score(conjunto[3], clf_boosting.predict_proba(conjunto[1])[:, 1]))
    metricas.append(row)
    
resultados = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score', 'auroc_score'])
resultados.sort_values(by='auroc_score', inplace=True, ascending=False)
resultados

,encoding,scaling,selection,score,auroc_score
4,onehot,normalizer,var_thres,0.813665,0.878951
2,onehot,robust,var_thres,0.844720,0.874796
0,onehot,standard,var_thres,0.844720,0.874633
1,onehot,minmax,var_thres,0.844720,0.871457
3,onehot,power,var_thres,0.844720,0.871131


#### 3.7.4 Stacking

In [46]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_1 = neighbors.KNeighborsClassifier()
    clf_2 = ensemble.RandomForestClassifier()
    stacker = ensemble.StackingClassifier(estimators=[('clf_1', clf_1), ('clf_2', clf_2)])
    
    stacker.fit(conjunto[0], conjunto[2])
    pred = stacker.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.819876
1,onehot,minmax,var_thres,0.819876
2,onehot,robust,var_thres,0.813665
3,onehot,power,var_thres,0.832298
4,onehot,normalizer,var_thres,0.807453


#### 3.7.5 Voting

In [47]:
clf_1 = neighbors.KNeighborsClassifier()
clf_2 = tree.DecisionTreeClassifier()
clf_3 = svm.SVC()

In [48]:
voting = ensemble.VotingClassifier(estimators=[
    ('clf_1', clf_1),
    ('clf_2', clf_2),
    ('clf_3', clf_3)
])

In [49]:
voting.fit(X_train_a, y_train_a)
pred = voting.predict(X_test_a)
score = (y_test_a == pred)
    
score.mean()

0.84472049689441

In [50]:
metricas = []

for conjunto in datos:
    row = []
    row.append(conjunto[4])
    row.append(conjunto[5])
    row.append(conjunto[6])
    
    clf_1 = neighbors.KNeighborsClassifier()
    clf_2 = tree.DecisionTreeClassifier()
    clf_3 = svm.SVC()
    voting = ensemble.VotingClassifier(estimators=[
        ('clf_1', clf_1),
        ('clf_2', clf_2),
        ('clf_3', clf_3)
    ])
    
    voting.fit(conjunto[0], conjunto[2])
    pred = voting.predict(conjunto[1])
    score = (conjunto[3] == pred)
    
    row.append(score.mean())
    metricas.append(row)
    
dd = pd.DataFrame(metricas, columns=['encoding','scaling','selection', 'score'])
dd

,encoding,scaling,selection,score
0,onehot,standard,var_thres,0.844720
1,onehot,minmax,var_thres,0.838509
2,onehot,robust,var_thres,0.826087
3,onehot,power,var_thres,0.850932
4,onehot,normalizer,var_thres,0.795031


#### 3.7.6 Cascading

In [85]:
#ver especificamente en otro momento